# Case2-3: CustomBusinessDayで満期日を列挙し、満期日までの日数を営業日で計算してみる

In [1]:
from itertools import chain
from datetime import datetime
from datetime import date
from pandas.tseries.offsets import CustomBusinessDay
import pandas as pd
import yaml
import requests

### 番外編
QuantLibに日本の祝日があるが、情報が古い

In [2]:
import QuantLib as ql
calender = ql.Japan()
calender.isHoliday(ql.Date(11, 8, 2016))

# 山の日がない！

False

* Windowsの場合はビルド済のwheelからインストールするのが簡単  
http://www.lfd.uci.edu/~gohlke/pythonlibs/#quantlib

## holiday_jpから祝日のyamlファイルを取得
> * Initial datasets
> 
> [komagata/holiday_jp](https://github.com/komagata/holiday_jp)
> 
> * komagata/holiday_jp Copyright
> 
> Copyright (c) 2009 Masaki Komagata. See [LICENSE](https://github.com/komagata/holiday_jp/blob/master/LICENSE) for details.
> 
> * "Datasets" idea
> 
> [Project Woothee](https://woothee.github.io/)

In [3]:
# https://github.com/k1LoW/holiday_jp
try:
    holiday_jp_yaml = yaml.load(requests.get('https://raw.githubusercontent.com/k1LoW/holiday_jp/master/holidays.yml').text)
except Exception:
    with open('data/holidays.yml', 'rb') as f:
        holiday_jp_yaml = yaml.load(f)

In [4]:
holiday_jp = list(holiday_jp_yaml.keys())

In [5]:
sorted(holiday_jp)[:5]

[datetime.date(1970, 1, 1),
 datetime.date(1970, 1, 15),
 datetime.date(1970, 2, 11),
 datetime.date(1970, 3, 21),
 datetime.date(1970, 4, 29)]

## 取引所は三が日と大晦日が休場のため休みを追加する

In [6]:
holiday_newyear = chain.from_iterable(
    [[date(y, 1, 2), date(y, 1, 3), date(y, 12, 31)]
     for y in range(sorted(holiday_jp)[0].year, sorted(holiday_jp)[-1].year + 1)])

In [7]:
holiday_jpx = chain(holiday_jp, holiday_newyear)

## CustomBusinessDayのインスタンスを作成

In [8]:
bday_jpx = CustomBusinessDay(holidays=holiday_jpx)

## Case2-1と同様にdate_rangeを作成
freq='WOM-2FRI'を指定することで毎月の第二金曜日を指定できる

In [9]:
sq_date = pd.date_range(datetime(2017, 1, 1), datetime(2017, 12, 31), freq='WOM-2FRI')

In [10]:
sq_date

DatetimeIndex(['2017-01-13', '2017-02-10', '2017-03-10', '2017-04-14',
               '2017-05-12', '2017-06-09', '2017-07-14', '2017-08-11',
               '2017-09-08', '2017-10-13', '2017-11-10', '2017-12-08'],
              dtype='datetime64[ns]', freq='WOM-2FRI')

## CustomBusinessDayで作成したインスタンスを利用して第二金曜日が祝日だった場合は一営業日前倒しする

In [11]:
def shift_bday(dt):
    if dt.date() in bday_jpx.holidays:
        return (dt - bday_jpx)
    else:
        return dt

In [12]:
sq_date = sq_date.map(shift_bday)

In [13]:
sq_date

array([Timestamp('2017-01-13 00:00:00', offset='WOM-2FRI'),
       Timestamp('2017-02-10 00:00:00', offset='WOM-2FRI'),
       Timestamp('2017-03-10 00:00:00', offset='WOM-2FRI'),
       Timestamp('2017-04-14 00:00:00', offset='WOM-2FRI'),
       Timestamp('2017-05-12 00:00:00', offset='WOM-2FRI'),
       Timestamp('2017-06-09 00:00:00', offset='WOM-2FRI'),
       Timestamp('2017-07-14 00:00:00', offset='WOM-2FRI'),
       Timestamp('2017-08-10 00:00:00'),
       Timestamp('2017-09-08 00:00:00', offset='WOM-2FRI'),
       Timestamp('2017-10-13 00:00:00', offset='WOM-2FRI'),
       Timestamp('2017-11-10 00:00:00', offset='WOM-2FRI'),
       Timestamp('2017-12-08 00:00:00', offset='WOM-2FRI')], dtype=object)

8月11日が祝日なので、一日前倒しされる

## 2017年1月SQから2月SQまでの営業日

In [14]:
pd.date_range(sq_date[0], sq_date[1], freq=bday_jpx)

DatetimeIndex(['2017-01-13', '2017-01-16', '2017-01-17', '2017-01-18',
               '2017-01-19', '2017-01-20', '2017-01-23', '2017-01-24',
               '2017-01-25', '2017-01-26', '2017-01-27', '2017-01-30',
               '2017-01-31', '2017-02-01', '2017-02-02', '2017-02-03',
               '2017-02-06', '2017-02-07', '2017-02-08', '2017-02-09',
               '2017-02-10'],
              dtype='datetime64[ns]', freq='C')

## 営業日をカウントすれば営業日数を得られる

In [15]:
len(pd.date_range(sq_date[0], sq_date[1], freq=bday_jpx))

21

## 番外編
pandas 0.18.1からCustomBusinessHourが使えるように  

http://pandas.pydata.org/pandas-docs/version/0.18.1/whatsnew.html#custom-business-hour

In [16]:
from pandas.tseries.offsets import CustomBusinessHour

### JPX(東証)の取引時間を設定

In [17]:
bhour_jpx = CustomBusinessHour(start='9:00', end='15:00', holidays=bday_jpx.holidays)

In [18]:
### 営業時間で計算ができる

In [19]:
datetime(2016, 9, 21, 13) + bhour_jpx * 2

Timestamp('2016-09-23 09:00:00')